# Setup

In [ ]:
!pip install --upgrade -q tcia-utils
!pip install -q pydicom
# !pip install imageio
!pip install scikit-learn

In [129]:
from pathlib import Path
import pandas as pd, numpy as np
from tqdm import tqdm
import pydicom as dicom
from io import BytesIO
import requests
from tcia_utils import nbia
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import math
from sklearn.model_selection import train_test_split
import shutil

In [140]:
__dir__ = Path().absolute().parent
data_dir =__dir__/"data" 
dcm_dir = data_dir/"dcm"
masked_data_dir = data_dir/"masked_cropped"
dataset_dir = data_dir/"dataset"
augmented_dir = data_dir/"augmented"

dcm_dir.mkdir(exist_ok=True)
masked_data_dir.mkdir(exist_ok=True)
augmented_dir.mkdir(exist_ok=True)
dataset_dir.mkdir(exist_ok=True)


dcm_extension = ".dcm"

In [114]:
mass_train = "https://wiki.cancerimagingarchive.net/download/attachments/22516629/mass_case_description_train_set.csv?version=1&modificationDate=1506796355038&api=v2"
calc_train = "https://wiki.cancerimagingarchive.net/download/attachments/22516629/calc_case_description_train_set.csv?version=1&modificationDate=1506796349666&api=v2"
calc_test = "https://wiki.cancerimagingarchive.net/download/attachments/22516629/calc_case_description_test_set.csv?version=1&modificationDate=1506796343686&api=v2"
mass_test = "https://wiki.cancerimagingarchive.net/download/attachments/22516629/mass_case_description_test_set.csv?version=1&modificationDate=1506796343175&api=v2"
df_mass_train = pd.read_csv(BytesIO(requests.get(mass_train).content))
df_mass_test = pd.read_csv(BytesIO(requests.get(mass_test).content))
df_calc_train = pd.read_csv(BytesIO(requests.get(calc_train).content))
df_calc_test = pd.read_csv(BytesIO(requests.get(calc_test).content))
df_mass, df_calc = pd.concat([df_mass_train, df_mass_test]), pd.concat([df_calc_train, df_calc_test])
df_orig = pd.concat([df_mass, df_calc])
df = df_orig.copy()

try:
  data = nbia.getSeries(collection = "CBIS-DDSM")
  df_meta_orig = nbia.getSeries(collection = "CBIS-DDSM", format="df")
  df_meta = df_meta_orig.copy()
  print(f"There are {len(data)} scans in total")
except TypeError:
  print("Server unavailable")

2024-06-02 19:14:35,455:INFO:Calling... https://services.cancerimagingarchive.net/nbia-api/services/v1/getSeries with parameters {'Collection': 'CBIS-DDSM'}
2024-06-02 19:14:44,705:INFO:Calling... https://services.cancerimagingarchive.net/nbia-api/services/v1/getSeries with parameters {'Collection': 'CBIS-DDSM'}


There are 6775 scans in total


# Functions

In [131]:
def download_patient(id):
  """download all data given patient id"""
  patient_dir = dcm_dir/id
  patient_dir.mkdir(exist_ok=True)
  series_uids = df_meta.query("PatientID_num == @id")['SeriesInstanceUID'].to_list()
  nbia.downloadSeries(series_uids, input_type = "list", path=str(patient_dir))
  return patient_dir

def read_dcm(file, plot=False):
    ds1 = dicom.dcmread(file)
    a = ds1.pixel_array

    if plot:
        plt.imshow(a, cmap='gray')
    return a

def get_cropped_img(full_img, mask):
    binary_mask = mask //255
    masked_image = full_img * binary_mask
    coords = np.column_stack(np.where(binary_mask == 1))
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)
    cropped_image = masked_image[x_min:x_max+1, y_min:y_max+1]
    return cropped_image

def show_dcm(patient, desc):

    df_patient = df_meta.query("PatientID_num ==@patient & SeriesDescription == @desc").copy()
    print(df_patient)
    for row in df_patient.iterrows():
        parent_dir = Path(row[1]["SeriesInstanceUID"])
        dcm_filenames = parent_dir.glob("*.dcm")
        for dcm_filename in dcm_filenames:
            dcm_file = parent_dir/dcm_filename
            print(dcm_file)
            read_dcm(dcm_file, True)


def clean_directory(path):
    """
    Recursively delete all files and subdirectories in a given directory.
    
    :param path: Pathlib Path object or string of the directory to clean.
    """
    dir_path = Path(path)
    if dir_path.exists() and dir_path.is_dir():
        for item in dir_path.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Recursively delete directory
            else:
                item.unlink()  # Delete file
        print(f"All contents removed from {dir_path}")
    else:
        print(f"The directory {dir_path} does not exist or is not a directory.")

# Preprocessing

In [116]:
# Remove unique value columns
df_meta = df_meta[df_meta.columns[df_meta.nunique() > 1]]

# mutual preprocessing
df_meta_orig = df_meta_orig[df_meta_orig.columns[df_meta_orig.nunique() > 1]]
df_meta_orig['PatientID_num'] = df_meta_orig['PatientID'].str.split('_').str[1:3].str.join('_')
df_meta_orig["abnormality_type"] = df_meta_orig['PatientID'].str.split('-').str[0]


df_meta['PatientID_num'] = df_meta['PatientID'].str.split('_').str[1:3].str.join('_')
df_meta_orig['PatientID_num'] = df_meta_orig['PatientID'].str.split('_').str[1:3].str.join('_')
df_meta["abnormality_type"] = df_meta['PatientID'].str.split('-').str[0]
df_meta_orig["abnormality_type"] = df_meta_orig['PatientID'].str.split('-').str[0]
df_meta = df_meta[df_meta['abnormality_type'] == "Mass"]
df_meta['PatientID'] = df_meta['PatientID'].str.split('_').str[3:].str.join("_") 
df_meta["abnorm_num"] = df_meta['PatientID'].str.split('_').str[2]
df_meta['abnorm_num'] = df_meta['abnorm_num'].fillna(1) # all NaN are only 1 abnormality
df_meta['PatientID'] = df_meta['PatientID'].str.split('_').str[0:2].str.join("_")

# fix duplicate column breast density
df.loc[df['breast_density'].isna(), 'breast_density'] = df.loc[~df['breast density'].isna(), 'breast density']
df['breast_density'] = df['breast_density'].astype('int32')

# fix column names
df_mass.columns, df_calc.columns  = df_mass.columns.str.replace(' ', '_'), df_calc.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace(' ', '_')



# mass shape limited to most common
mass_shape_val = ['OVAL', 'IRREGULAR', 'LOBULATED', 'ROUND']
df = df.query("mass_shape in @mass_shape_val")

# assessment limited to 0,3,4,5
birads_val = [0, 3, 4, 5]
df = df.query("assessment in @birads_val")

# create masked_cropped column for aiding in dataset split
df["masked_cropped"] = df["patient_id"] + "_" + "Mass" + "_" + df["left_or_right_breast"] + "_" + df["image_view"] + "_" +df["abnormality_id"].astype('str')

# Benign without callback = benign
df["pathology"] = df["pathology"].replace(["BENIGN_WITHOUT_CALLBACK"], "BENIGN")

# drop unused columns
df_meta = df_meta.drop(columns=["StudyInstanceUID", "TimeStamp", "FileSize", "abnormality_type"])
df = df.drop(columns=["image_file_path","cropped_image_file_path", "ROI_mask_file_path"])

df = df[~df['patient_id'].isin(["P_00016"])] # doesn't have ROI images

In [117]:
# save for future use
df_meta.to_csv(data_dir/'df_meta.csv')
df.to_csv(data_dir/"df.csv")

# Discrepencies 

1. No mention of clip limit values
2. They state that CBIS-DDSM  "presents cases in BI-RADS category 2 to 5"
3. Do they use the cropped dataset due to mass shape values also in BiRADS and pathology classification, or the full(+calcification) dataset?
5. BiRADS distribution significantly different. Their category 2 is considered as 0.
6. "we resize the detected and segmented ROIs from 256 × 256..."

### Number of patients

In [81]:
print(f"unique mass patients with {mass_shape_val} mass shapes: {df_orig.loc[df_orig['mass shape'].isin(mass_shape_val), 'patient_id'].nunique()}") 
print(f"total #patients (+calc): {df_orig['patient_id'].nunique()}")
print(f"Stated #patients: 1555")


unique mass patients with ['OVAL', 'IRREGULAR', 'LOBULATED', 'ROUND'] mass shapes: 778
total #patients (+calc): 1566
Stated #patients: 1555


### mass lesions mammograms

In [90]:
# all types of mass lesions
print(f"  ours: {df_mass['image_file_path'].nunique()}\nTheirs: 1467")

  ours: 1592
Theirs: 1467


### Pathology

In [96]:
df_orig["pathology"].value_counts()

MALIGNANT                  1457
BENIGN                     1429
BENIGN_WITHOUT_CALLBACK     682
Name: pathology, dtype: int64

In [91]:
df_augmented["pathology"].value_counts()

BENIGN       4644
MALIGNANT    3900
Name: pathology, dtype: int64

which | Benign    |Malignant |
------|-----------|----------|
Ours  | 4644      | 3900     |
Theirs| 4500      | 4302     |

In [27]:
df["assessment"].value_counts()

4    590
3    331
5    317
0    156
2     27
1      3
Name: assessment, dtype: int64

### mass shape

In [92]:
df_augmented["mass_shape"].value_counts()

IRREGULAR    2784
OVAL         2472
LOBULATED    2304
ROUND         984
Name: mass_shape, dtype: int64

which | IRREGULAR|  OVAL     |LOBULATED |    ROUND |
------|----------|-----------|----------|----------|
Ours  | 2784      | 2472     | 2304     | 984      | 
Theres| 3846      | 2040     |  2112    | 804     |   

### BiRads

In [94]:
df_augmented["assessment"].value_counts()

4    3540
3    1986
5    1902
0     936
2     162
1      18
Name: assessment, dtype: int64

which | 2        |    3     |        4 |        5 |        6 |
------|----------|----------|----------|----------|----------|
Ours  | 162      | 1986     | 3540     | 1902     | 0        |
Theres| 792      | 1938     |  2328    | 3402     | 0        |


# Get all mass masked images

In [147]:
# patients = cropped_list
problematic_lst = []
new_list = []
all_patients_mass = list(df_meta['PatientID_num'].unique())
patients = all_patients_mass
# patients = ["P_00016"]
print(f"proccesing patients {patients[0]} to {patients[-1]}...")
for patient in tqdm(patients, total=len(patients)):
    print(f"patient {patient}:")
    # if patient not in cropped_list:
    download_patient(patient)
    df_abnorm = df_meta.loc[(df_meta['PatientID_num'] == patient) & (df_meta['SeriesDescription']=='ROI mask images')]
    

    for row in df_abnorm.iterrows():
        out = None
        abnormality_type, scan_type=row[1]["abnormality_type"], row[1]["PatientID"]
        abnorm = row[1]["abnorm_num"]
        abnorm_folder_name = row[1]["SeriesInstanceUID"]

        df_full = df_meta[(df_meta["PatientID_num"] == patient) & (df_meta['PatientID'] == scan_type) & (df_meta['SeriesDescription'] == 'full mammogram images')]
        if len(df_full) !=1:
            problematic_lst.append(patient)
            break
        full_folder_name = df_full.iloc[0]['SeriesInstanceUID']

        full_dcm = dcm_dir/patient/full_folder_name/"1-1.dcm"
        full_pixel = read_dcm(full_dcm)
        # print(full_pixel.shape)
        abnorm_folder = dcm_dir/patient/abnorm_folder_name
        abnorm_dcm_files = list(abnorm_folder.rglob("*.dcm"))
        if len(abnorm_dcm_files)>1:
            a, b = read_dcm(abnorm_dcm_files[0]), read_dcm(abnorm_dcm_files[1])
            mask = a if a.shape > b.shape else b
        else:
            mask = read_dcm(abnorm_dcm_files[0])
        padding = [(0, a_dim - b_dim) for a_dim, b_dim in zip(full_pixel.shape, mask.shape)]
        mask_padded = np.pad(mask, padding, mode='constant', constant_values=0)
    # for abnorm_dcm in abnorm_dcm_files:
    #     abnorm_pixel = read_dcm(abnorm_dcm)
        # print(b.shape)
        
    # if full_pixel.shape == abnorm_pixel.shape:
        out = get_cropped_img(full_pixel, mask_padded)
        new_name = patient + '_' + abnormality_type + '_' + scan_type + '_' + abnorm + '.png'
        img_path = masked_data_dir/new_name
        print("--------------------")
        print(img_path)
        print("---------------------")
        cv2.imwrite(str(img_path), out)
                # break
        
        if out is None:
            print(f"failed for {patient}")
            new_list.append(patient)

proccesing patients P_01009 to P_01912...


  0%|          | 0/892 [00:00<?, ?it/s]2024-06-02 20:37:54,364:INFO:Downloading 4 Series Instance UIDs (scans).
2024-06-02 20:37:54,365:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.117041576511324414842508325652101471266 already downloaded.
2024-06-02 20:37:54,366:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.261488099311480249940936912860154272271 already downloaded.
2024-06-02 20:37:54,367:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.366380199312149445220426062372662360356 already downloaded.
2024-06-02 20:37:54,368:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.423381440611407931423565073193041412507 already downloaded.
2024-06-02 20:37:54,369:INFO:Downloaded 0 out of 4 Series Instance UIDs (scans).
0 failed to download.
4 previously downloaded.


patient P_01009:
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01009_Mass_RIGHT_CC_1.png
---------------------


  0%|          | 1/892 [00:00<03:20,  4.45it/s]2024-06-02 20:37:54,589:INFO:Downloading 4 Series Instance UIDs (scans).
2024-06-02 20:37:54,590:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.43873839610761788013224723323225482381 already downloaded.
2024-06-02 20:37:54,590:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.323819613710302263925884159030090866040 already downloaded.
2024-06-02 20:37:54,591:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.51941006212124694604546402963586764653 already downloaded.
2024-06-02 20:37:54,592:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.148086015311392424203944628703986309919 already downloaded.
2024-06-02 20:37:54,592:INFO:Downloaded 0 out of 4 Series Instance UIDs (scans).
0 failed to download.
4 previously downloaded.


--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01009_Mass_RIGHT_MLO_1.png
---------------------
patient P_00061:
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_00061_Mass_RIGHT_MLO_1.png
---------------------


...

--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01807_Mass_RIGHT_MLO_1.png
---------------------
patient P_01815:
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01815_Mass_RIGHT_CC_1.png
---------------------
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01815_Mass_RIGHT_MLO_1.png
---------------------
patient P_01825:


2024-06-02 20:40:36,399:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.404577305912728645825627508702429716956 already downloaded.
2024-06-02 20:40:36,400:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.356066733313901824110606752460413887917 already downloaded.
2024-06-02 20:40:36,401:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.408076999112697002131052158793819620269 already downloaded.
2024-06-02 20:40:36,402:INFO:Downloaded 0 out of 4 Series Instance UIDs (scans).
0 failed to download.
4 previously downloaded.
100%|█████████▉| 889/892 [02:42<00:00,  4.50it/s]2024-06-02 20:40:36,572:INFO:Downloading 2 Series Instance UIDs (scans).
2024-06-02 20:40:36,573:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.355464339112205472025411629030840677852 already downloaded.
2024-06-02 20:40:36,574:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.183250739910949037237308684010112484798 already downloaded.
2024-06-02 20:40:36,575:INFO:Downloaded 0 out of 2 Series Instance UIDs (scans).
0 failed to download.
2 previously downloaded.
20

--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01825_Mass_RIGHT_CC_1.png
---------------------
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01825_Mass_RIGHT_MLO_1.png
---------------------
patient P_01833:
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01833_Mass_RIGHT_MLO_1.png
---------------------
patient P_01865:


100%|█████████▉| 891/892 [02:42<00:00,  6.34it/s]2024-06-02 20:40:36,737:INFO:Downloading 4 Series Instance UIDs (scans).
2024-06-02 20:40:36,738:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.395806490413060973910077403213561423281 already downloaded.
2024-06-02 20:40:36,739:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.17600404511466616531880746830203874983 already downloaded.
2024-06-02 20:40:36,739:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.299479312912648479419761596652152845359 already downloaded.
2024-06-02 20:40:36,740:WARNING:Series 1.3.6.1.4.1.9590.100.1.2.424889390711092892018162257160068574314 already downloaded.
2024-06-02 20:40:36,741:INFO:Downloaded 0 out of 4 Series Instance UIDs (scans).
0 failed to download.
4 previously downloaded.
100%|██████████| 892/892 [02:42<00:00,  5.49it/s]

--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01865_Mass_LEFT_MLO_1.png
---------------------
patient P_01912:
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01912_Mass_RIGHT_CC_1.png
---------------------
--------------------
/home/kostya/projects/CBIS-DDSM/data/masked_cropped/P_01912_Mass_RIGHT_MLO_1.png
---------------------


# Build Datasets for models

## Preprocessing images

In [ ]:
# Histogram Equalization
# 

## Augmentation

In [21]:
rot_angles = [90, 180, 270]
clip_limit_values = [2, 10]
tileGridSize=(8, 8)

In [ ]:
img_path = masked_data_dir/"P_00001_Mass_LEFT_CC_1_90.png"
image_cv2 = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
b = cv2.equalizeHist(image_cv2)

In [148]:
clean_directory(augmented_dir)
src_paths = list(masked_data_dir.glob("*.png"))
for img_path in tqdm(src_paths, total=len(src_paths), desc="Augmenting x6..."):
    with Image.open(img_path) as img:
        for rot_angle in rot_angles:
            img_rot = img.rotate(rot_angle, expand=True)
            dest_path = augmented_dir/(img_path.stem + f"_{str(rot_angle)}.png")
            img_rot.save(dest_path)

        for clip_value in clip_limit_values:
            img_np = np.array(img)
            img_np = img_np.astype(np.uint16)
            clahe = cv2.createCLAHE(clipLimit=clip_value, tileGridSize=tileGridSize)
            clahe_image = clahe.apply(img_np)
            clahe_pil_image = Image.fromarray(clahe_image)

            dest_path = augmented_dir/(img_path.stem + f"_clahe{clip_value}.png")
            clahe_pil_image.save(dest_path)
        dest_file = augmented_dir/img_path.name
        shutil.copy2(img_path, dest_file)

All contents removed from /home/kostya/projects/CBIS-DDSM/data/augmented


Augmenting x6...: 100%|██████████| 1694/1694 [01:59<00:00, 14.13it/s]


## create augmented df

In [118]:
df_augmented = df.copy()
for rot_angle in rot_angles:
    df_rot = df.copy()
    df_rot['masked_cropped'] = df_rot['masked_cropped'] + f'_{str(rot_angle)}'
    df_augmented = pd.concat([df_augmented, df_rot], ignore_index=True)

for clip_value in clip_limit_values:
    df_clahe = df.copy()
    df_clahe['masked_cropped'] = df_clahe['masked_cropped'] + f'_clahe{clip_value}'
    df_augmented = pd.concat([df_augmented, df_clahe], ignore_index=True)

# Split train, val, test

In [155]:
train_size = 0.8
val_size = 0.5
seed = 42

In [139]:
for file_path in masked_data_dir.glob("*clahe*.png"):
    file_path.unlink()  # Delete the file

for rot_angle in rot_angles:
    for file_path in masked_data_dir.glob(f"*{rot_angle}*.png"):
        file_path.unlink()  # Delete the file

In [156]:
clean_directory(dataset_dir)
df_augmented = df_augmented.sample(frac=1, random_state=seed).reset_index(drop=True)
for task in ["assessment", "pathology", "mass_shape"]:
    train_df, temp_df = train_test_split(df_augmented, train_size=train_size, random_state=seed, stratify=df_augmented[task])
    val_df, test_df = train_test_split(temp_df, train_size=val_size, random_state=seed, stratify=temp_df[task])
    print(f"working on {task}...")
    for df_type, split in zip([train_df, val_df, test_df], ['train', 'val', 'test']):
        
        for row in tqdm(df_type.iterrows(), total=len(df_type), desc=f"{split} processing..."):
            filename = row[1]["masked_cropped"] + ".png"
            src_file = augmented_dir/filename

            dest_dir = dataset_dir/task/split/str(row[1][task])
            dest_dir.mkdir(parents=True, exist_ok=True)
            dest_file = dest_dir/ src_file.name
            shutil.copy2(src_file, dest_file)

All contents removed from /home/kostya/projects/CBIS-DDSM/data/dataset
working on assessment...


test processing...: 100%|██████████| 836/836 [00:00<00:00, 1136.94it/s]


working on pathology...


test processing...: 100%|██████████| 836/836 [00:00<00:00, 3271.52it/s]


working on mass_shape...


test processing...: 100%|██████████| 836/836 [00:00<00:00, 956.21it/s] 


# MISC

In [92]:
!rm -r ../data/masked_cropped/

In [106]:
!mkdir ../data/dataset
!mkdir ../data/dataset/birads
!mkdir ../data/dataset/birads/train
!mkdir ../data/dataset/birads/val
!mkdir ../data/dataset/birads/test

mkdir: cannot create directory ‘../data/dataset’: File exists
mkdir: cannot create directory ‘../data/dataset/birads’: File exists


# Conclusions

1. `cv2` imwrite reduces uint 16 to uint8, use `cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)`
2. `imageio` retain resolution, PIL converts to comfortalbe one (like uint16 -> int32)

In [175]:
import torch.optim as optim
import torch.nn as nn

In [181]:
model = nn.Linear(10, 2)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, verbose=True)

for i in range(25):
    print('Epoch ', i)
    scheduler.step(1.)    
    # print(optimizer.param_groups[0]['lr'])
    print(f"lr = {scheduler.get_last_lr()[0]}")

Epoch  0
lr = 0.001
Epoch  1
lr = 0.001
Epoch  2
lr = 0.001
Epoch  3
lr = 0.001
Epoch  4
lr = 0.001
Epoch  5
lr = 0.001
Epoch  6
lr = 0.0001
Epoch  7
lr = 0.0001
Epoch  8
lr = 0.0001
Epoch  9
lr = 0.0001
Epoch  10
lr = 0.0001
Epoch  11
lr = 0.0001
Epoch  12
lr = 1e-05
Epoch  13
lr = 1e-05
Epoch  14
lr = 1e-05
Epoch  15
lr = 1e-05
Epoch  16
lr = 1e-05
Epoch  17
lr = 1e-05
Epoch  18
lr = 1.0000000000000002e-06
Epoch  19
lr = 1.0000000000000002e-06
Epoch  20
lr = 1.0000000000000002e-06
Epoch  21
lr = 1.0000000000000002e-06
Epoch  22
lr = 1.0000000000000002e-06
Epoch  23
lr = 1.0000000000000002e-06
Epoch  24
lr = 1.0000000000000002e-07
